## Nimeüksused lausetes

In [1]:
# Vajalikud impordid
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from collections import Counter
import itertools
import csv
from copy import deepcopy

### I Andmete sisselugemine andmebaasist

Tühi järjend lausete jaoks:

In [2]:
corpus = []

Tühi järjend lausete ID-de jaoks:

In [3]:
sentence_ids = []

Andmebaasist loetakse sisse laused, mis sisaldavad mõnda NerTaggeri poolt peale märgitud nimeüksust:

In [4]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [5]:
for row in cur.execute("SELECT ID, sentence FROM sentences WHERE named_entity = 1"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    corpus.append(sentence)
    sentence_ids.append(sentence_id)

In [6]:
con.close()

In [7]:
len(corpus)

39314

### II Nimeüksuste korrigeerimine

Vaatame üht lauset ja selle nimeüksusi:

In [10]:
for ner in corpus[0].ner:
    print(ner.nertag, ner.text, ner.enclosing_text)

LOC ['Tartus'] Tartus
ORG ['Rein'] Rein
LOC ['Tallinnas'] Tallinnas
PER ['Hans'] Hans
ORG ['H'] H


NerTagger võib olla teinud vigu, näiteks märkinud isikunime asukohaks, asukoha organisatsiooniks jne. Siin katsetatakse lihtsakoelist lahendust, mis ei korrigeeri kindlasti kõiki selliseid vigu, kuid võib nende vigade hulka vähendada. Nimelt loetakse kokku eri nimeüksuste esinemised kõigis liikides. Kui nimeüksus esineb rohkem kui ühes liigis, loetakse õigeks liigiks see, milles tema esinemissagedus kõige suurem on. Kui esinemissagedused on eri liikides võrdsed, ei tehta aga praeguses etapis midagi.

Esiteks luuakse tühjad järjendid nii isikunimede, organisatsiooninimede kui asukohanimede jaoks:

In [11]:
entities_PER = []
entities_ORG = []
entities_LOC = []

Seejärel käiakse läbi kõik korpuses olevad laused ja lisatakse nimeüksused järjenditesse vastavalt nende liigile:

In [12]:
for sentence in corpus:
    for ner in sentence.ner:
        lemmatized_ner = [word.lemma[0] for word in ner]
        if ner.nertag == 'PER':
            entities_PER.append(' '.join(lemmatized_ner))
        elif ner.nertag == 'ORG':
            entities_ORG.append(' '.join(lemmatized_ner))
        else:
            entities_LOC.append(' '.join(lemmatized_ner))

In [13]:
# kui palju on isikunime märgendi saanud üksusi
len(entities_PER)

28303

In [14]:
# kui palju on organisatsiooninime märgendi saanud üksusi
len(entities_ORG)

20147

In [15]:
# kui palju on asukohanime märgendi saanud üksusi
len(entities_LOC)

37524

Seejärel tuleb tekitada kõigi nimeolemiliikide sagedusloendid:

In [16]:
entities_PER_lower = [name.lower() for name in entities_PER]
entities_ORG_lower = [name.lower() for name in entities_ORG]
entities_LOC_lower = [name.lower() for name in entities_LOC]

In [17]:
frequencies_PER = Counter(entities_PER_lower)
frequencies_ORG = Counter(entities_ORG_lower)
frequencies_LOC = Counter(entities_LOC_lower)

In [18]:
#Kümme sagedasemat isiku-, organisatsiooni- ja asukohanime
print('PER' + '\t' + 'ORG' + '\t' + 'LOC')
for (per, org, loc) in itertools.zip_longest(frequencies_PER.most_common(10), frequencies_ORG.most_common(10), frequencies_LOC.most_common(10)):
    print(per, org, loc)

PER	ORG	LOC
('priit hõbemägi', 732) ('keskerakond', 2286) ('eesti', 10696)
('edgar savisaar', 717) ('reformierakond', 2075) ('tallinn', 2894)
('savisaar', 648) ('riigikogu', 1540) ('euroopa', 1680)
('siim kallas', 439) ('irl', 1042) ('venemaa', 1461)
('jüri ratas', 416) ('postimees', 594) ('tartu', 1098)
('ainar ruussaar', 415) ('nato', 438) ('soome', 933)
('mart luik', 334) ('euroopa liit', 335) ('euroopa liit', 759)
('ansip', 303) ('keskpäevatund', 266) ('eesti vabariik', 662)
('hannes astok', 257) ('eesti energia', 244) ('vene', 574)
('jürgen ligi', 253) ('tallinn sadam', 205) ('läti', 496)


Luuakse uued tühjad järjendid isiku-, organisatsiooni- ja asukohanimede jaoks, mida võib hiljem kasutada nimede mugavaks andmebaasi lisamiseks.

In [19]:
entities_PER_new = []
entities_ORG_new = []
entities_LOC_new = []

Sagedusloenditest tehakse koopiad, sest originaalloenditest hakatakse üksusi eemaldama:

In [20]:
frequencies_PER_copy = deepcopy(frequencies_PER)
frequencies_ORG_copy = deepcopy(frequencies_ORG)
frequencies_LOC_copy = deepcopy(frequencies_LOC)

Tühi järjend nimeüksustele, mis esinevad rohkem kui ühes klassis võrdsel määral:

In [21]:
equal_occurrences = []

In [22]:
#Kas isikunimede hulgas olevad nimed leiduvad ka organisatsiooni- või asukohanimede hulgas:
for name in frequencies_PER_copy.keys():
    if frequencies_ORG_copy[name] > 0:
        if frequencies_ORG_copy[name] > frequencies_PER_copy[name]:
            #print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_ORG_copy[name] < frequencies_PER_copy[name]:
            #print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_ORG[name]
        else:
            equal_occurrences.append(name)
                
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_PER_copy[name]:
            #print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_LOC_copy[name] < frequencies_PER_copy[name]:
            #print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Kas organisatsiooninimede hulgas leidub nimesid, mis esinevad ka asukohanimede hulgas (isikunimesid pole vaja uuesti läbi käia):
for name in frequencies_ORG_copy.keys():
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_ORG_copy[name]:
            #print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_ORG[name]
        elif frequencies_LOC_copy[name] < frequencies_ORG_copy[name]:
            #print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Asukohanimesid pole vaja eraldi läbi käia, seal leiduvad duplikaadid kas isiku- või organisatsiooninimedest on juba tuvastatud.  

In [25]:
# millised nimeüksused rohkem kui ühes klassis võrdsel määral esinesid
equal_occurrences = set(equal_occurrences)
print(equal_occurrences)

{'saksa bundestag', 'taxify', 'kiisa', 'hong', 'coca-cola', 'breivik', 'vikipeedia', 'rotermanni', 'swedbankis', 'dudajev', 'paul-eerik', 'helsinki', 'enron', 'real', 'expo', 'mäkinen', 'riia riia', 'martin helmes', 'katainen', 'postimehes', 'camorra', 'matsin', 'donatra', 'boroditš', 'dante', 'epp tohver', 'midfield', 'hansson', 'harju', 'service', 'france', 'fifa', 'magnitsk', 'just', 'saara', 'l. ron', 'madison', 'dressen', 'abbas', 'berlingo', 'maarja vaino', 'sotši', 'muammar gaddaf', 'gatesi', 'tõnisson', 'vaikma', 'stolitsa', 'prada', 'tapa', 'loeb', 'kisel', 'napoleon', 'eerik', 'sik', 'vaba', 'nord streami', 'rooma', 'lemett', 'kukerpill', 'reinsalule', 'nissan', 'pr', 'warhol', 'rossi', 'honda crv', 'ecker', 'tel', 'uuber', 'ekre isamaa', 'nissan qashqai', 'kalev meedia', 'rain lõhmus', 'transparency', 'neeme raud', 'ekrest', 'alex', 'rose', 'vladimir svet', 'enef', 'balticconnector', 'sults', 'järvikule', 'tori', 'sotsiaaldemokraatlik', 'kreenholm', 'facebook facebook', 'jaa

Võrdsel määral rohkem kui ühes klassis nimeüksuseid võib soovi korral käsitsi kureerida või määrata neile klass automaatselt, valides juhuslikult NerTaggeri poolt pakutud variantide vahel. Siinses töös nendega aga rohkem edasi ei toimetata.

### III Nimeüksuste salvestamine

Saadud parandustega nimeüksuste loendid tuleb salvestada, et neid saaks edasises töös kasutada. Siin salvestatakse nimeüksused nii CSV-faili (võimaldab käsitsi muudatuste tegemist) kui SQLite andmebaasi (võimaldab neid hiljem mugavalt sisse lugeda).

Läbitöödeldud sagedusloendite nimeüksused kantakse vastavatesse järjenditesse:

In [26]:
# neid järjendeid kasuatakse nimeüksuste andmebaasi salvestamisel
for name in frequencies_PER.keys():
    entities_PER_new.append(name)
    
for name in frequencies_ORG.keys():
    entities_ORG_new.append(name)
    
for name in frequencies_LOC.keys():
    entities_LOC_new.append(name)

Saadud parandustega sagedusloendid talletatakse csv-failis, et võimaldada vajadusel käsitsi korrektuuride tegemist.

In [27]:
with open('ner_frequencies.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['name', 'freq', 'value'])
    writer.writeheader()
    header2 = {'name': 'string', 'freq': 'integer', 'value': 'string'}
    writer.writerow(header2)
    for name in frequencies_PER.keys():
        writer.writerow({'name': name, 'freq': frequencies_PER[name], 'value': 'PER'})
    for name in frequencies_ORG.keys():
        writer.writerow({'name': name, 'freq': frequencies_ORG[name], 'value': 'ORG'})
    for name in frequencies_LOC.keys():
        writer.writerow({'name': name, 'freq': frequencies_LOC[name], 'value': 'LOC'})

In [28]:
with open('ner_frequencies.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [29]:
#Esimesed 10 rida
print(rows[:10])

[{'name': 'string', 'freq': 'integer', 'value': 'string'}, {'name': 'hans', 'freq': '15', 'value': 'PER'}, {'name': 'eerik', 'freq': '9', 'value': 'PER'}, {'name': 'andrus ansip', 'freq': '242', 'value': 'PER'}, {'name': 'meelis', 'freq': '35', 'value': 'PER'}, {'name': 'olev esula', 'freq': '1', 'value': 'PER'}, {'name': 'vadim', 'freq': '2', 'value': 'PER'}, {'name': 'juskin', 'freq': '1', 'value': 'PER'}, {'name': 'margus', 'freq': '11', 'value': 'PER'}, {'name': 'sven', 'freq': '31', 'value': 'PER'}]


Huvipakkuvad nimeüksused (siinses töös isiku- ja organisatsiooninimed) talletatakse lisaks ka andmebaasis. Rohkem kui ühes klassis võrdsel määral esinenud nimeüksused filtreeritakse ühtlasi selle käigus välja:

In [30]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [31]:
cur.execute("CREATE TABLE named_entities(ID INTEGER PRIMARY KEY, sentence_ID INTEGER, PER TEXT, ORG TEXT)")

In [32]:
for i in range(len(corpus)):
    for ner in corpus[i].ner:
        # salvestatakse lemmad
        lemmatized_ner = [word.lemma[0] for word in ner]
        lemmatized_ner_text = ' '.join(lemmatized_ner).lower()
        if ner.nertag == 'PER' and lemmatized_ner_text in entities_PER_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'PER' and lemmatized_ner_text in entities_PER_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag == 'ORG' and lemmatized_ner_text in entities_ORG_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'ORG' and lemmatized_ner_text in entities_ORG_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        con.commit()

In [36]:
con.close()